In [22]:
import os
import numpy as np
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [88]:
class VolleyballDataset(Dataset):
    def __init__(self, root_dir, annotation_dir, transform=None):
        self.root_dir = root_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.video_dirs = sorted([int(d) for d in os.listdir(root_dir) if d.isdigit()])
        self.sequence_len = 41
        
    def __len__(self):
        return 55
    
    def __getitem__(self, idx):
        video_idx = idx // self.sequence_len
        sequence_idx = idx % self.sequence_len
        video_dir = os.path.join(self.root_dir, str(self.video_dirs[video_idx]))
        sequence_file = os.path.join(self.annotation_dir, str(self.video_dirs[video_idx]), str(sequence_idx)+'.txt')
        print(video_dir, sequence_file)
        with open(sequence_file, 'r') as f:
            line = f.readline()
            while line.startswith('0 0'):
                line = f.readline()
            x, y = [float(coord) for coord in line.split()]
        img_file = os.path.join(video_dir, '{:05d}.png'.format(sequence_idx))
        img = Image.open(img_file)
        if self.transform:
            img = self.transform(img)
        target = {'boxes': torch.tensor([[x-5, y-5, x+5, y+5]]), 'labels': torch.tensor([1])}
        return img, target

root_dir = 'C:/Users/salba/Documents/videos'
annotation_dir = 'C:/Users/salba/Documents/volleyball_ball_annotation'
dataset = VolleyballDataset(root_dir, annotation_dir, transform=torchvision.transforms.ToTensor())

train_videos = [1, 3, 6, 7, 10, 13, 15, 16, 18, 22, 23, 31, 32, 36, 38, 39, 40, 41, 42, 48, 50, 52, 53, 54]
val_videos = [0, 2, 8, 12, 17, 19, 24, 26, 27, 28, 30, 33, 46, 49, 51]
test_videos = [4, 5, 9, 11, 14, 20, 21, 25, 29, 34, 35, 37, 43, 44, 45, 47]

train_idx = [i for i in range(len(dataset)) if dataset.video_dirs[i] in train_videos]
val_idx = [i for i in range(len(dataset)) if dataset.video_dirs[i] in val_videos]
test_idx = [i for i in range(len(dataset)) if dataset.video_dirs[i] in test_videos]

train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)
test_dataset = torch.utils.data.Subset(dataset, test_idx)

In [59]:
print('Number of train samples:', len(train_dataset))
print('Number of val samples:', len(val_dataset))
print('Number of test samples:', len(test_dataset))

Number of train samples: 24
Number of val samples: 15
Number of test samples: 16


In [86]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    losses = []
    header = 'Epoch: [{}]'.format(epoch)
    for i, (images, targets) in enumerate(data_loader):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses.append(sum(loss for loss in loss_dict.values()))

        optimizer.zero_grad()
        losses[-1].backward()
        optimizer.step()

        if i % print_freq == 0:
            print(f"Iteration {i}/{len(data_loader)}, Loss = {losses[-1]}")

    return sum(losses) / len(losses)

In [89]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
import torchvision.utils as utils
import torchvision.models.detection as detection

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

num_classes = 2  # ball and background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=torch.utils.data.dataloader.default_collate)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=torch.utils.data.dataloader.default_collate)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=torch.utils.data.dataloader.default_collate)

# Train the model for 10 epochs
num_epochs = 10
for epoch in range(num_epochs):
    # Train for one epoch, printing every 10 iterations
    train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)
    # Update the learning rate scheduler
    lr_scheduler.step()
    # Evaluate on the validation set
    evaluate(model, val_data_loader, device=device)


C:/Users/salba/Documents/videos\0 C:/Users/salba/Documents/volleyball_ball_annotation\0\31.txt


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/salba/Documents/volleyball_ball_annotation\\0\\31.txt'